In [0]:
!pip install langchain langchain-groq langchain-community pyspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 108.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 609.9/609.9 kB 75.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Not uninstalling requests at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-67805deb-0145-43b8-a258-940d08fc3de7
    Can't uninstall 'requests'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import os
from langchain_groq import ChatGroq
from langchain_community.utilities import SparkSQL
from langchain_classic.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# 1. Setup API Key
os.environ["GROQ_API_KEY"] = ""  # Replace with your Groq Key

# Initialize the LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile", 
    temperature=0
)

In [0]:
# Creates a "Virtual" Database representation of my Gold Tables
# We only expose the specific tables we want the AI to query
catalog = "spotify_cata"
schema = "gold"
tables = ["fact_streams", "dim_artists", "dim_tracks"]

# LangChain Spark SQL wrapper
spark_sql = SparkSQL(schema=schema, catalog=catalog)
spark_sql.dialect = 'spark'

In [0]:
from langchain_classic.chains import create_sql_query_chain
from langchain_core.prompts import PromptTemplate

# Custom prompt that instructs LLM to return only SQL
custom_prompt = PromptTemplate.from_template(
    """Given an input question, create a syntactically correct {dialect} query to run.
    
Only use the following tables:
{table_info}

Question: {input}

Return ONLY the SQL query without any explanation, markdown formatting, or additional text.
SQL Query:"""
)

# 1. Chain that generates the SQL query with custom prompt
write_query = create_sql_query_chain(llm, spark_sql, prompt=custom_prompt)

# 2. Function that executes the SQL query using Spark
def execute_query(query):
    result = spark.sql(query.strip())
    return result.toPandas().to_string()

# 3. Combine them: Write Query -> Execute Query -> Answer
chain = write_query | execute_query

In [0]:
# Question 1
response = chain.invoke({"question": "Who are the top 3 artists by total streams?"})
print(response)

        artist_name  total_streams
0    Kirsten Harris             13
1      Kendra Bates             11
2  Jennifer Montoya             10


In [0]:
# Question 2 (Complex Join)
response = chain.invoke({"question": "What is the average duration of songs in the Pop genre?"})
print(response)

   average_duration
0        212.715789
